In [ ]:
def take_genes_from_bed(name_of_bed: str):
    names = ['chr', 'start', 'stop', 'gene', 'to_drop', 'strand']
    df = pd.read_csv(name_of_bed, sep="\t",
                     header=None,
                     names=names,
                     skiprows=1)

    df = df.drop('to_drop', axis=1)
    return np.unique(df.gene.values)

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
names = ['chr', 'position', 'coverage']
names2 = ['chr', 'start', 'stop', 'gene', 'to_drop', 'strand']

name_of_bed = 'DHS-003Z.primers-150bp.bed'

depth = pd.read_csv('samtools_depth', sep="\t", names=names)
primers = pd.read_csv(name_of_bed, sep="\t",
                     header=None,
                     names=names2,
                     skiprows=1)

df2 = df2.drop('to_drop', axis=1)

In [ ]:
np.diff(take_genes_from_bed(name_of_bed), np.unique(df.gene.values))

In [ ]:
df.head()

In [ ]:
genes_bound = dict()
for gene in take_genes_from_bed(name_of_bed):
    df3 = df2[df2['gene'] == gene]
    genes_bound[gene] = (df3.iloc[0]['start'],
                         df3.iloc[len(df3)-1]['stop'])
print(genes_bound)

    

In [ ]:
def add_gene_name(df_to_add, df_from_add): # TODO OPTIMIZE
    genes = np.array(['0' * 10 for i in range(len(df_to_add))])
    for pos in range(len(df_to_add)):
        for index, row in df_from_add.iterrows():
            if (row['start'] + 1 <= df_to_add.position[pos] <= row['stop']) and\
                    (df_to_add.chr[pos] == row['chr']):
                genes[pos] = row['gene']b
                break
    return genes 

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
file_name = 'samtools_output2'
names = ["chr", "start", "end", "gene", "to_drop", "strand", "depth"]
df = pd.read_csv(file_name, sep="\t", names=names).drop("to_drop", axis=1)

In [ ]:
df.head()

In [ ]:
# df.depth.sum() / 
(df.end - df.start).sum()

In [ ]:
def genes_depth():
    dct = {"genes": [], 
            "cov": []}
    
    for gene in np.unique(df.gene.values):
        dct["genes"].append(gene)
        dct["cov"].append(df[df.gene == gene].depth.sum() / 
                        (df[df.gene == gene].end - df[df.gene == gene].start).sum())
        
    return pd.DataFrame(dct)
    

In [ ]:
df2 = genes_depth()

In [ ]:
df2.sort_values('cov', ascending=False)

In [ ]:
import pandas as pd
import numpy as np

In [116]:
file_name = 'samtools_output2'
names = ["chr", "start", "end", "gene", "to_drop", "strand", "depth"]
df = pd.read_csv(file_name, sep="\t", names=names).drop("to_drop", axis=1)

In [ ]:
df.head()

In [ ]:
# df.depth.sum() / 
(df.end - df.start).sum()

In [ ]:
def genes_depth(df):
    dct = {"gene": [], 
            "mean_cov_depth_X": []}
    
    for gene in np.unique(df.gene.values):
        dct["gene"].append(gene)
        dct["mean_cov_depth_X"].append(df[df.gene == gene].depth.sum() / 
                        (df[df.gene == gene].end - df[df.gene == gene].start).sum())
    
    result = pd.DataFrame(dct)
    
    result = result.sort_values('mean_cov_depth_X', ascending=False) \
    .reset_index() \
    .drop('index', axis=1)
    
    result["mean_cov_depth_X"] = result["mean_cov_depth_X"].apply(round)
    
#     result['mean_cov_depth_X'] = df['mean_cov_depth_X'].apply(round)
    
    return result
    

In [ ]:
df2 = genes_depth(df)

In [ ]:
df2

In [ ]:
df2 = df2.sort_values('cov', ascending=False).reset_index().drop('index', axis=1)

In [ ]:
df2['cov'] = df2['cov'].apply(round)

In [ ]:
df2.rename({"cov": "mean_cov_depth_X"}, axis=1)

In [114]:
import io
import subprocess
import pandas as pd
import numpy as np
import tempfile



def depth_calculation_in_target_regions(target_regions: str = target_regions_,
                                        patient: str = patient_) -> pd.DataFrame:
    command = f"samtools bedcov {target_regions} {patient}".strip().split(' ')
    temp_file = tempfile.NamedTemporaryFile()

    samtools_bedcov = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, encoding="utf-8")

    with open(temp_file.name, 'w')as temp:
        temp.write(samtools_bedcov.stdout)

    if not samtools_bedcov.stderr:
        names = ["chr", "start", "end", "gene", "to_drop", "strand", "depth"]
        df = pd.read_csv(temp_file, sep="\t", names=names).drop("to_drop", axis=1)

        return df

    raise PipelineError(f"an error occurred during the\n >>>> {' '.join(command)} <<<<\nERROR from {samtools_bedcov.stderr}")



df2 = depth_calculation_in_target_regions()

In [119]:
df

,chr,start,end,gene,strand,depth
0,chr1,12918805,12918926,PRAMEF2,+,0
1,chr1,12918861,12918973,PRAMEF2,+,0
2,chr1,12918942,12919066,PRAMEF2,+,0
3,chr1,12918987,12919110,PRAMEF2,+,0
4,chr1,12919089,12919215,PRAMEF2,-,0
5,chr1,12919534,12919661,PRAMEF2,-,0
6,chr1,12919840,12919952,PRAMEF2,+,0
7,chr1,12919681,12919798,PRAMEF2,-,0
8,chr1,12919910,12920033,PRAMEF2,+,0
9,chr1,12919755,12919873,PRAMEF2,-,0


In [ ]:
(df == df2).sum()

chr       False
start     False
end       False
gene      False
strand    False
depth     False
dtype: bool